 # Interfaz de PyQt con numeros y graficas

In [ ]:
import sys                                                                                                  # Importo el módulo sys, necesario para acceder a funcionalidades del sistema
from sympy import Matrix, cos, sin
import numpy as np                                                                                          # Importo numpy, una biblioteca para operaciones numéricas en Python
import math                                                                                                 # Importo el módulo math para usar funciones matemáticas básicas
from math import radians, degrees, cos, sin                                                                 # Importo funciones específicas del módulo math: radians, degrees, cos, y sin
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QHBoxLayout, QLabel, QLineEdit, QPushButton, QMainWindow # Importo las clases necesarias de PyQt5 para crear la interfaz gráfica
import matplotlib.pyplot as plt
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas                            # Importo las clases necesarias para integrar gráficos de matplotlib en la interfaz de PyQt5
from matplotlib.figure import Figure                                                                        # Importo la clase Figure de matplotlib, que se usa para crear figuras de gráficos
import serial
import time


class MplCanvas(FigureCanvas):                                                                              # Defino una clase MplCanvas que hereda de FigureCanvas para gestionar gráficos en 3D
    def __init__(self, parent=None):
        fig = Figure()                                                                                      # Creo una figura de matplotlib para el gráfico
        self.ax = fig.add_subplot(111, projection='3d')                                                     # Agrego un subplot 3D a la figura
        super().__init__(fig)                                                                               # Inicializo el canvas con la figura creada

class KinematicsApp(QWidget):                                                                               # Defino la clase principal de la aplicación KinematicsApp, que hereda de QWidget
    def __init__(self):
        super().__init__()                                                                                  # Llamo al constructor de la clase base QWidget
        self.initUI()                                                                                       # Inicializo la interfaz gráfica
        self.serial_port = None  # Inicializa el puerto serie como None

    def initUI(self):

        self.setStyleSheet("""
            QWidget {
                background-color: black; 
                color: white; 
            }
            QLabel {
                font-size: 14px; 
            }
            QLineEdit {
                background-color: #333333; 
                color: white; 
                border: 1px solid #555555; 
            }
            QPushButton {
                background-color: #00bfff; 
                color: black; 
                border: 2px solid #00bfff; 
                border-radius: 15px; 
                padding: 10px; 
                font-size: 14px; 
            }
            QPushButton:hover {
                background-color: #1e90ff; 
            }
            QPushButton:pressed {
                background-color: #4682b4; 
            }
        """)

        self.setWindowTitle('Kinematics Calculator')                                                        # Establezco el título de la ventana de la aplicación

        main_layout = QHBoxLayout()                                                                         # Creo un layout horizontal que contendrá todos los elementos de la interfaz
        
        direct_layout = QVBoxLayout()                                                                       # Configuro el layout para la sección de cinemática directa
        direct_layout.addWidget(QLabel('Cinemática Directa'))                                               # Agrego un texto en la interfaz para indicar que esta sección es para cinemática directa

        self.q1_label = QLabel('Q1:')                                                                       # Creo una etiqueta que dice "Q1:" 
        self.q1_input = QLineEdit(self)                                                                     # Creo un campo de texto donde el usuario puede ingresar el valor de Q1
        direct_layout.addWidget(self.q1_label)                                                              # Agrego la etiqueta de Q1  
        direct_layout.addWidget(self.q1_input)                                                              # Agrego el campo de texto al layout para colocar el valor de Q1 

        self.q2_label = QLabel('Q2:')                                                                       # Creo una etiqueta que dice "Q2:" 
        self.q2_input = QLineEdit(self)                                                                     # Creo un campo de texto donde el usuario puede ingresar el valor de Q2
        direct_layout.addWidget(self.q2_label)                                                              # Agrego la etiqueta de Q2  
        direct_layout.addWidget(self.q2_input)                                                              # Agrego el campo de texto al layout para colocar el valor de Q2

        self.q3_label = QLabel('Q3:')                                                                       # Creo una etiqueta que dice "Q3:" 
        self.q3_input = QLineEdit(self)                                                                     # Creo un campo de texto donde el usuario puede ingresar el valor de Q3
        direct_layout.addWidget(self.q3_label)                                                              # Agrego la etiqueta de Q3  
        direct_layout.addWidget(self.q3_input)                                                              # Agrego el campo de texto al layout para colocar el valor de Q3

        self.q4_label = QLabel('Q4:')                                                                       # Creo una etiqueta que dice "Q4:" 
        self.q4_input = QLineEdit(self)                                                                     # Creo un campo de texto donde el usuario puede ingresar el valor de Q4
        direct_layout.addWidget(self.q4_label)                                                              # Agrego la etiqueta de Q4  
        direct_layout.addWidget(self.q4_input)                                                              # Agrego el campo de texto al layout para colocar el valor de Q4
        
        self.calculate_direct_button = QPushButton('Calcular Cinemática Directa', self)                     # Creo un botón que dice "Calcular Cinemática Directa". Este botón ejecutará la función para calcular la cinemática directa cuando se haga clic en él
        self.calculate_direct_button.clicked.connect(self.calculate_direct_kinematics)                      # Conecto el botón a la función que realizará el cálculo de la cinemática directa
        direct_layout.addWidget(self.calculate_direct_button)                                               # Agrego el botón al layout de cinemática directa

        self.direct_result_label = QLabel('Resultados Directos:', self)                                     # Creo una etiqueta para mostrar los resultados de la cinemática directa
        direct_layout.addWidget(self.direct_result_label)                                                   # Agrego la etiqueta al layout de cinemática directa

        self.canvas_direct = MplCanvas(self)                                                                # Inicializo un área de gráficos para mostrar la visualización de los resultados de cinemática directa
        direct_layout.addWidget(self.canvas_direct)                                                         # Agrego el área de gráficos al layout de cinemática directa

        # Adding the new button for direct kinematics
        self.send_direct_button = QPushButton('Enviar a Arduino - Directa', self)
        self.send_direct_button.setStyleSheet("background-color: #9370db; border-radius: 15px;padding: 10px;font-size: 14px;QPushButton:hover {background-color: #8b7b8b;}QPushButton:pressed {background-color: #663399;}")
        self.send_direct_button.clicked.connect(self.send_direct_to_arduino)
        direct_layout.addWidget(self.send_direct_button)

        inverse_layout = QVBoxLayout()                                                                      # Configuro el layout para la sección de cinemática inversa
        inverse_layout.addWidget(QLabel('Cinemática Inversa'))                                              # Agrego un texto en la interfaz para indicar que esta sección es para cinemática inversa

        self.px_label = QLabel('Px:')                                                                       # Creo una etiqueta que dice "Px:"  
        self.px_input = QLineEdit(self)                                                                     # Creo un campo de texto donde el usuario puede ingresar el valor de Px
        inverse_layout.addWidget(self.px_label)                                                             # Agrego la etiqueta de Px 
        inverse_layout.addWidget(self.px_input)                                                             # Agrego el cammpo de texto al layout de cinemática inversa donde se ingresa el valor de Px

        self.py_label = QLabel('Py:')                                                                       # Creo una etiqueta que dice "Py:"  
        self.py_input = QLineEdit(self)                                                                     # Creo un campo de texto donde el usuario puede ingresar el valor de Py
        inverse_layout.addWidget(self.py_label)                                                             # Agrego la etiqueta de Py 
        inverse_layout.addWidget(self.py_input)                                                             # Agrego el cammpo de texto al layout de cinemática inversa donde se ingresa el valor de Py

        self.pz_label = QLabel('Pz:')                                                                       # Creo una etiqueta que dice "Pz:"  
        self.pz_input = QLineEdit(self)                                                                     # Creo un campo de texto donde el usuario puede ingresar el valor de Pz
        inverse_layout.addWidget(self.pz_label)                                                             # Agrego la etiqueta de Pz 
        inverse_layout.addWidget(self.pz_input)                                                             # Agrego el cammpo de texto al layout de cinemática inversa donde se ingresa el valor de Pz

        self.pitch_label = QLabel('Pitch:')                                                                 # Creo una etiqueta que dice "Pitch:"  
        self.pitch_input = QLineEdit(self)                                                                  # Creo un campo de texto donde el usuario puede ingresar el valor de Pitch
        inverse_layout.addWidget(self.pitch_label)                                                          # Agrego la etiqueta de Pitch 
        inverse_layout.addWidget(self.pitch_input)                                                          # Agrego el cammpo de texto al layout de cinemática inversa donde se ingresa el valor de Pitch

        self.calculate_inverse_button = QPushButton('Calcular Cinemática Inversa', self)                    # Creo un botón que dice "Calcular Cinemática Inversa". Este botón ejecutará la función para calcular la cinemática inversa cuando se haga clic en él
        self.calculate_inverse_button.clicked.connect(self.calculate_inverse_kinematics)                      # Conecto el botón a la función que realizará el cálculo de la cinemática directa
        inverse_layout.addWidget(self.calculate_inverse_button)                                             # Agrego el botón al layout de cinemática inversa
        
        self.inverse_result_label = QLabel('Resultados Inversos:', self)                                    # Creo una etiqueta para mostrar los resultados de la cinemática inversa
        inverse_layout.addWidget(self.inverse_result_label)                                                 # Agrego la etiqueta al layout de cinemática inversa

        self.canvas_inverse = MplCanvas(self)                                                               # Inicializo un área de gráficos para mostrar la visualización de los resultados de cinemática inversa
        inverse_layout.addWidget(self.canvas_inverse)                                                       # Agrego el área de gráficos al layout de cinemática inversa

        # Adding the new button for inverse kinematics
        self.send_inverse_button = QPushButton('Enviar a Arduino - Inversa', self)
        self.send_inverse_button.setStyleSheet("background-color: #9370db; border-radius: 15px;padding: 10px;font-size: 14px;QPushButton:hover { background-color: #8b7b8b; }QPushButton:pressed { background-color: #663399; }")
        self.send_inverse_button.clicked.connect(self.send_inverse_to_arduino)
        inverse_layout.addWidget(self.send_inverse_button)

        # Etiqueta de confirmación para cinemática directa
        self.confirmation_label_direct = QLabel('', self)
        direct_layout.addWidget(self.confirmation_label_direct)

        # Etiqueta de confirmación para cinemática inversa 
        self.confirmation_label_inverse = QLabel('', self)
        inverse_layout.addWidget(self.confirmation_label_inverse)
                         
        main_layout.addLayout(direct_layout)                                                                # Agrego el layout de cinematica directa al layout principal
        main_layout.addLayout(inverse_layout)                                                               # Agrego el layout de cinematica inversa al layout principal
        self.setLayout(main_layout)                                                                         # Establezco el layout principal para la ventana, que contiene ambos layouts

#######
    
    def connect_serial(self, port='COM4', baudrate=9600):
            """Conecta al puerto serie de Arduino."""
            try:
                self.serial_port = serial.Serial(port, baudrate)
                time.sleep(2)  # Espera a que se establezca la conexión
            except Exception as e:
                print(f'Error al conectar con Arduino: {e}')

    def send_direct_to_arduino(self):
        """Envía los valores de entrada a Arduino para cinemática directa."""
        if self.serial_port is None:
            self.connect_serial()  # Conectar si no está conectado

        try:
            letter = 'D'  # Asumiendo que la letra es fija
            Q1 = float(self.q1_input.text())
            Q2 = float(self.q2_input.text())
            Q3 = float(self.q3_input.text())
            Q4 = float(self.q4_input.text())

            # Formatear los datos para enviar
            data = f"{letter},{Q1:07.2f},{Q2:07.2f},{Q3:07.2f},{Q4:07.2f}\n"
            self.serial_port.write(data.encode('utf-8'))  # Enviar datos a Arduino
            print(f'Datos enviados a Arduino: {data.strip()}')
            # Actualizar la etiqueta de confirmación
            self.confirmation_label_direct.setText('Se han enviado los valores a Arduino.')
        except Exception as e:
            print(f'Error al enviar datos: {e}')

    def send_inverse_to_arduino(self):
        """Envía los valores de entrada a Arduino para cinemática inversa."""
        if self.serial_port is None:
            self.connect_serial()  # Conectar si no está conectado

        try:
            letter = 'I'  # Asumiendo que la letra es fija
            Px = float(self.px_input.text())
            Py = float(self.py_input.text())
            Pz = float(self.pz_input.text())
            pitch = float(self.pitch_input.text())

            # Formatear los datos para enviar
            data = f"{letter},{Px:07.2f},{Py:07.2f},{Pz:07.2f},{pitch:07.2f}\n"
            self.serial_port.write(data.encode('utf-8'))  # Enviar datos a Arduino
            print(f'Datos enviados a Arduino: {data.strip()}')
            # Actualizar la etiqueta de confirmación
            self.confirmation_label_inverse.setText('Se han enviado los valores a Arduino.')
        except Exception as e:
            print(f'Error al enviar datos: {e}')
#######

    def calculate_direct_kinematics(self):
        try:
            Q1 = radians(float(self.q1_input.text()))                                                       # Convierto el valor de Q1 del cuadro de entrada a un número flotante y luego a radianes
            Q2 = radians(float(self.q2_input.text()))                                                       # Convierto el valor de Q2 del cuadro de entrada a un número flotante y luego a radianes
            Q3 = radians(float(self.q3_input.text()))                                                       # Convierto el valor de Q3 del cuadro de entrada a un número flotante y luego a radianes
            Q4 = radians(float(self.q4_input.text()))                                                       # Convierto el valor de Q4 del cuadro de entrada a un número flotante y luego a radianes

            Px, Py, Pz, pitch_codo_arriba,pitch_codo_abajo = self.plot_3D_cinematica_directa(Q1, Q2, Q3, Q4, self.canvas_direct)                                                            # Llamo a la función plot_robot_arm para calcular Px, Py, Pz y los ángulos de pitch del brazo robótico
            self.direct_result_label.setText(f'Resultados Directos:\nPx: {Px}\nPy: {Py}\nPz: {Pz}\nPitch codo arriba: {pitch_codo_arriba}\nPitch codo Abajo:{pitch_codo_abajo}')            # Muestro los resultados en la etiqueta de resultados directos
        except ValueError:
            self.direct_result_label.setText('Por favor, ingresa valores numéricos válidos.')                                                                                               # Si los valores ingresados no son números válidos, muestro un mensaje de error                         

    def calculate_inverse_kinematics(self):
        try:
            Px = float(self.px_input.text())                                                                        # Convierto el valor de Px del cuadro de entrada a un número flotante
            Py = float(self.py_input.text())                                                                        # Convierto el valor de Py del cuadro de entrada a un número flotante
            Pz = float(self.pz_input.text())                                                                        # Convierto el valor de Pz del cuadro de entrada a un número flotante
            pitch = float(self.pitch_input.text())                                                                  # Convierto el valor de pitch del cuadro de entrada a un número flotante

            Q1, Q2, Q3, Q4 = self.plot_3D_cinematica_inversa(Px, Py, Pz, pitch, self.canvas_inverse)                # Llamo a la función plot_inverse_kinematics para calcular los ángulos Q1, Q2, Q3 y Q4 del brazo robótico
            self.inverse_result_label.setText(f'Resultados Inversos:\nQ1: {Q1}\nQ2: {Q2}\nQ3: {Q3}\nQ4: {Q4}')      # Muestro los resultados en la etiqueta de resultados inversos
        except ValueError:
            self.inverse_result_label.setText('Por favor, ingresa valores numéricos válidos.')                      # Si los valores ingresados no son números válidos, muestro un mensaje de error

    def plot_3D_cinematica_directa(self, Q1, Q2, Q3, Q4, canvas):

        l1 = 96.000                                                                                         # Longitud l1 en milímetros
        l2 = 13.90                                                                                          # Longitud l2 en milímetros
        l3 = 150                                                                                            # Longitud l3 en milímetros
        l4 = 143.24                                                                                         # Longitud l4 en milímetros
        l5 = 127.42                                                                                         # Longitud l5 en milímetros

        A0_1 = Matrix([[cos(Q1), 0, sin(Q1), l2*cos(Q1)],                                                           # Matriz de transformación desde la base hasta la primera articulación
                    [sin(Q1), 0, -cos(Q1), l2*sin(Q1)],
                    [0, 1, 0, l1],
                    [0, 0, 0, 1]])

        A1_2 = Matrix([[cos(Q2), -sin(Q2), 0, l3*cos(Q2)],                                                          # Matriz de transformación desde la primera hasta la segunda articulación
                    [sin(Q2), cos(Q2), 0, l3*sin(Q2)],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])

        A2_3 = Matrix([[cos(Q3), -sin(Q3), 0, l4*cos(Q3)],                                                          # Matriz de transformación desde la segunda hasta la tercera articulación
                    [sin(Q3), cos(Q3), 0, l4*sin(Q3)],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])

        A3_4 = Matrix([[cos(Q4), -sin(Q4), 0, l5*cos(Q4)],                                                          # Matriz de transformación desde la tercera hasta el efector
                    [sin(Q4), cos(Q4), 0, l5*sin(Q4)],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])

        A0_2 = A0_1 * A1_2                                                                                          # Calculo la transformación acumulada desde la base hasta la articulacion 2
        A0_3 = A0_2 * A2_3                                                                                          # Calculo la transformación acumulada desde la base hasta la articulacion 3
        A0_4 = A0_3 * A3_4                                                                                          # Calculo la transformación acumulada desde la base hasta el efector

        coords_A0_1 = A0_1[:3, 3]                                                                                   # Obtengo el vector de posicion que va desde la base hasta la articulacion 1
        coords_A0_2 = A0_2[:3, 3]                                                                                   # Obtengo el vector de posicion que va desde la base hasta la articulacion 2                                                       
        coords_A0_3 = A0_3[:3, 3]                                                                                   # Obtengo el vector de posicion que va desde la base hasta la articulacion 3
        coords_A0_4 = A0_4[:3, 3]                                                                                   # Obtengo el vector de posicion que va desde la base hasta el efector

        canvas.ax.cla()                                                                                             # Limpio el gráfico anterior en el lienzo
        
        canvas.ax.scatter(0, 0, 0, color='k', label='Articulacion 1')                                               # Grafico articulación 1 en el origen del espacio 3D
        canvas.ax.scatter(coords_A0_1[0], coords_A0_1[1], coords_A0_1[2], color='g', label='Articulación 2')        # Grafico articulación 2 en el espacio 3D
        canvas.ax.scatter(coords_A0_2[0], coords_A0_2[1], coords_A0_2[2], color='b', label='Articulación 3')        # Grafico articulación 3 en el espacio 3D
        canvas.ax.scatter(coords_A0_3[0], coords_A0_3[1], coords_A0_3[2], color='r', label='Articulación 4')        # Grafico articulación 4 en el espacio 3D
        canvas.ax.scatter(coords_A0_4[0], coords_A0_4[1], coords_A0_4[2], color='m', label='Efector')               # Grafico efector final en el espacio 3D

        x = [0, A0_1[0, 3], A0_2[0, 3], A0_3[0, 3], A0_4[0, 3]]                                                     # Coordenadas X para graficar las lineas entre las articulaciones
        y = [0, A0_1[1, 3], A0_2[1, 3], A0_3[1, 3], A0_4[1, 3]]                                                     # Coordenadas Y para graficar las lineas entre las articulaciones
        z = [0, A0_1[2, 3], A0_2[2, 3], A0_3[2, 3], A0_4[2, 3]]                                                     # Coordenadas Z para graficar las lineas entre las articulaciones

        Q1_deg = math.degrees(Q1)                                                                                   # Convierto el angulo Q1 a grados
        Q2_deg = math.degrees(Q2)                                                                                   # Convierto el angulo Q2 a grados
        Q3_deg = math.degrees(Q3)                                                                                   # Convierto el angulo Q3 a grados
        Q4_deg = math.degrees(Q4)                                                                                   # Convierto el angulo Q4 a grados

        canvas.ax.text(0, 0, 0, f'Q1: {Q1_deg:.2f}°', color='black', fontsize=6)                                                                                        # Añado texto con el valor del angulo Q1 en grados
        canvas.ax.text(coords_A0_1[0], coords_A0_1[1], coords_A0_1[2], f'Q2: {Q2_deg:.2f}°', color='green', fontsize=6)                                                 # Añado texto con el valor del angulo Q2 en grados
        canvas.ax.text(coords_A0_2[0], coords_A0_2[1], coords_A0_2[2], f'Q3: {Q3_deg:.2f}°', color='blue', fontsize=6)                                                  # Añado texto con el valor del angulo Q3 en grados
        canvas.ax.text(coords_A0_3[0], coords_A0_3[1], coords_A0_3[2], f'Q4: {Q4_deg:.2f}°', color='red', fontsize=6)                                                   # Añado texto con el valor del angulo Q4 en grados
        canvas.ax.text(coords_A0_4[0], coords_A0_4[1], coords_A0_4[2], f'({A0_4[0, 3]:.1f}, {A0_4[1, 3]:.1f}, {A0_4[2, 3]:.1f}) mm', color='magenta', fontsize=6)       # Añado texto con el valor de la coordenada del efector final

        canvas.ax.set_xlim(-400, 400)                                                           # Límite del eje X
        canvas.ax.set_ylim(-400, 400)                                                           # Límite del eje Y
        canvas.ax.set_zlim(0, 400)                                                              # Límite del eje Z

        canvas.ax.plot(x, y, z, label="Cinematica del brazo robotico 4GDL")                     # Dibujo las líneas que representan el brazo robótico (Conectan las articulaciones)
        canvas.ax.set_xlabel('Eje X')                                                           # Etiqueta del eje X
        canvas.ax.set_ylabel('Eje Y')                                                           # Etiqueta del eje Y
        canvas.ax.set_zlabel('Eje Z')                                                           # Etiqueta del eje Z
        canvas.ax.legend(fontsize=6)                                                            # Leyenda del gráfico

        canvas.draw()                                                                           # Dibujo el gráfico actualizado

        Px = A0_4[0, 3]                                                                         # Extraigo las coordenadas en X del efector final
        Py = A0_4[1, 3]                                                                         # Extraigo las coordenadas en Y del efector final
        Pz = A0_4[2, 3]                                                                         # Extraigo las coordenadas en Z del efector final
        pitch_codo_arriba = degrees(abs(abs(Q4)-abs(Q2)+abs(Q3)))                               # Ángulo de pitch para el codo hacia arriba
        pitch_codo_abajo = (abs(-abs(degrees(Q4))+abs(degrees(Q2))-abs(degrees(Q3))-360))       # Ángulo de pitch para el codo hacia abajo

        return Px, Py, Pz, pitch_codo_arriba,pitch_codo_abajo                                   # La cinematica directa me devuelve las coordenadas del efector final y los ángulos de pitch
    
    def Cinematica_Inversa(self,Px4,Py4,Pz4,Q):                                                             # Si o si la tengo que definir dentro de la clase KinematicsApp

        np.set_printoptions(suppress=True, precision=2)                                                     # Configuro las opciones de impresión de numpy para suprimir notación científica y definir la precisión a 2 decimales

        l1 = 96.000                                                                                         # Longitud l1 en milímetros
        l2 = 13.90                                                                                          # Longitud l2 en milímetros
        l3 = 150                                                                                            # Longitud l3 en milímetros
        l4 = 143.24                                                                                         # Longitud l4 en milímetros
        l5 = 127.42                                                                                         # Longitud l5 en milímetros

        #Px4 = float(input("Colocar la posicion en X en mm que deseas:"))                                   # Solicito al usuario que ingrese el valor de 'Px' y lo convierto a un número flotante   
        #Py4 = float(input("Colocar la posicion en Y en mm que deseas:"))                                   # Solicito al usuario que ingrese el valor de 'Py' y lo convierto a un número flotante
        #Pz4 = float(input("Colocar la posicion en Z en mm que deseas:"))                                   # Solicito al usuario que ingrese el valor de 'Pz' y lo convierto a un número flotante
        #Q = float(input("Colocar angulo de cabeceo (o pitch) Q en grados: "))                              # Solicito al usuario que ingrese el valor del 'Angulo de pitch' y lo convierto a un número flotante

        print("\nLa posicion en XYZ ingresados manualmente son:\n", Px4,"mm",Py4,"mm",Pz4,"mm",Q,"°")       # Imprimo las posiciones ingresadas manualmente

        Qrad = math.radians(Q)                                                                              # Convierto el ángulo Q de grados a radianes

        ht = math.sqrt(Px4**(2) + Py4**(2))                                                                 # Calculo la hipotenusa ht usando las posiciones X e Y
        print("El valor de ht es:",ht)

        l5x = l5*cos(Qrad)                                                                                  # Calculo las componentes x y z de l5
        l5z = l5*sin(Qrad)

        h = math.sqrt((ht-l5x-l2)**(2) + (Pz4+l5z-l1)**(2))                                                 # Calculo la longitud h 
        print("El valor de h es:",h)

        print("\nargumento de a1:",(Pz4+l5z-l1)/(ht-l5x-l2))
        a1 = math.atan2((Pz4+l5z-l1), (ht-l5x-l2))                                                          # Calculo de los angulos incognitos del dibujo a1,a2,a3
        print ("El angulo a1 es:", round(math.degrees(a1), 6), "°\n")

        print("Argumento de Q1",math.atan2(Py4,Px4))
        Q1 = abs(math.atan2(Py4,Px4))                                                                       # Calculo el ángulo Q1 en radianes usando las posiciones X e Y

        q3_argument = (h**2 - l3**2 - l4**2) / (2*l3*l4)                                                    # Calculo el ángulo Q3 en radianes
        if abs(q3_argument) < 1:
            q3_root = math.sqrt(1 - q3_argument**2)
        else:
            q3_root = 0                                                                                     # Manejo de errores numéricos de la maquina
        Q3 = -abs(math.atan2(q3_root, q3_argument))

        print("argumento de a2:",((l4*sin(Q3))/(l3+l4*cos(Q3))))
        a2 = math.atan2( (l4*sin(Q3)),(l3+l4*cos(Q3)))                                                      # Calculo el ángulo a2 usando Q3
        print ("El angulo a2 es:", round(math.degrees(a2), 6), "°\n")

        Q2 = abs(a1) + abs(a2)                                                                              # Calculo el ángulo Q2 sumando a1 y a2

        Q4 = -abs(Q2) + abs(Q3) - abs(Qrad)                                                                 # Calculo el ángulo Q4 ajustando signos para obtener el valor correcto

        Q1_grados = math.degrees(Q1)                                                                        # Convierto angulo Q1 a grados
        Q2_grados = math.degrees(Q2)                                                                        # Convierto angulo Q2 a grados
        Q3_grados = math.degrees(Q3)                                                                        # Convierto angulo Q3 a grados
        Q4_grados = math.degrees(Q4)                                                                        # Convierto angulo Q4 a grados
        print ("\nEl angulo Q1 es:\n\n",round(Q1_grados,2),"°" )                                            # Imprimo el angulo Q1 en grados con dos decimales
        print ("\nEl angulo Q2 es:\n\n",round(Q2_grados,2),"°" )                                            # Imprimo el angulo Q2 en grados con dos decimales
        print ("\nEl angulo Q3 es:\n\n",round(Q3_grados,2),"°" )                                            # Imprimo el angulo Q3 en grados con dos decimales
        print ("\nEl angulo Q4 es:\n\n",round(Q4_grados,2),"°" )                                            # Imprimo el angulo Q4 en grados con dos decimales

        # ----------------------------- [Visualizacion en 3D] ------------------------------ #

        A0_1 = Matrix([[cos(Q1) , 0, sin(Q1)        , l2*cos(Q1)],                                          # Defino la matriz de transformación homogénea de la base al primer eslabón (obtenida por el metodo de Denavit-Hartenberg)
                    [sin(Q1) , 0, -cos(Q1)       , l2*sin(Q1)],
                    [0       , 1,    0           ,    l1     ],
                    [0       , 0,    0           ,    1      ]])

        A1_2 = Matrix([[cos(Q2) , -sin(Q2)  , 0     , l3*cos(Q2)],                                          # Defino la matriz de transformación homogénea del primer al segundo eslabón (obtenida por el metodo de Denavit-Hartenberg)
                    [sin(Q2) , cos(Q2)   , 0     , l3*sin(Q2)],
                    [0       ,    0      , 1     ,    0      ],
                    [0       ,    0      , 0     ,    1      ]])

        A2_3 = Matrix([[cos(Q3)  , -sin(Q3) , 0     , l4*cos(Q3)],                                          # Defino la matriz de transformación homogénea del segundo al tercer eslabón (obtenida por el metodo de Denavit-Hartenberg)
                    [sin(Q3)  , cos(Q3)  , 0     , l4*sin(Q3)],
                    [0        ,    0     , 1     ,     0     ],
                    [0        ,    0     , 0     ,     1     ]])

        A3_4 = Matrix([[cos(Q4) , -sin(Q4)  , 0     , l5*cos(Q4)],                                          # Defino la matriz de transformación homogénea del tercer al cuarto eslabón (obtenida por el metodo de Denavit-Hartenberg)
                    [sin(Q4) , cos(Q4)   , 0     , l5*sin(Q4)],
                    [0       ,   0       , 1     ,     0     ],
                    [0       ,   0       , 0     ,     1     ]])

        A0_2 = A0_1*A1_2                                                                                    # Calculo la matriz de transformación A0_2 multiplicando A0_1 y A1_2
        A0_3 = A0_2*A2_3                                                                                    # Calculo la matriz de transformación A0_3 multiplicando A0_2 y A2_3
        A0_4 = A0_3*A3_4                                                                                    # Calculo la matriz de transformación A0_4 multiplicando A0_3 y A3_4

        fig = plt.figure()                                                                                  # Creo una nueva figura para la visualización 3D
        ax = fig.add_subplot(111, projection='3d')                                                          # Añadir un subplot 3D a la figura.  111 significa que se está creando una cuadrícula de subplots de 1 fila y 1 columna, y se está seleccionando el primer (y único) subplot en esta cuadrícula. Básicamente, esto crea un solo subplot que ocupa toda la figura.

        coords_A0_1 = A0_1[:3, 3]                                                                           # Extraigo las coordenadas x, y, z de la matricez de transformación A0_1 (osea el vector de posicion)
        coords_A0_2 = A0_2[:3, 3]                                                                           # Extraigo las coordenadas x, y, z de la matricez de transformación A0_2 (osea el vector de posicion)
        coords_A0_3 = A0_3[:3, 3]                                                                           # Extraigo las coordenadas x, y, z de la matricez de transformación A0_3 (osea el vector de posicion) 
        coords_A0_4 = A0_4[:3, 3]                                                                           # Extraigo las coordenadas x, y, z de la matricez de transformación A0_4 (osea el vector de posicion)

        ax.scatter(0, 0, 0, color='k', label='Articulacion 1')                                              # Ploteo la primera articulación en el origen (0,0,0) con color negro
        ax.scatter(coords_A0_1[0], coords_A0_1[1], coords_A0_1[2], color='g', label='Articulación 2')       # Ploteo la segunda articulación en las coordenadas del primer segmento con color verde
        ax.scatter(coords_A0_2[0], coords_A0_2[1], coords_A0_2[2], color='b', label='Articulación 3')       # Ploteo la tercera articulación en las coordenadas del segundo segmento con color azul
        ax.scatter(coords_A0_3[0], coords_A0_3[1], coords_A0_3[2], color='r', label='Articulación 4')       # Ploteo la cuarta articulación en las coordenadas del tercer segmento con color rojo
        ax.scatter(coords_A0_4[0], coords_A0_4[1], coords_A0_4[2], color='m', label='Efector')              # Ploteo la posición del efector final con color magenta

        x = [0, A0_1[0,3], A0_2[0,3], A0_3[0,3], A0_4[0,3]]                                                 # Defino las coordenadas x de cada articulación y del efector
        y = [0, A0_1[1,3], A0_2[1,3], A0_3[1,3], A0_4[1,3]]                                                 # Defino las coordenadas y de cada articulación y del efector
        z = [0, A0_1[2,3], A0_2[2,3], A0_3[2,3], A0_4[2,3]]                                                 # Defino las coordenadas z de cada articulación y del efector

        ax.text(0, 0, 0, f'Q1: {Q1_grados:.2f}°', color='black', fontsize=6)                                                            # Agrego texto anotado en la primera articulación con el valor de Q1 en grados
        ax.text(coords_A0_1[0], coords_A0_1[1], coords_A0_1[2], f'Q2: {Q2_grados:.2f}°', color='green', fontsize=6)                     # Agrego texto anotado en la segunda articulación con el valor de Q2 en grados
        ax.text(coords_A0_2[0], coords_A0_2[1], coords_A0_2[2], f'Q3: {Q3_grados:.2f}°', color='blue', fontsize=6)                      # Agrego texto anotado en la tercera articulación con el valor de Q3 en grados
        ax.text(coords_A0_3[0], coords_A0_3[1], coords_A0_3[2], f'Q4: {Q4_grados:.2f}°', color='red', fontsize=6)                       # Agrego texto anotado en la cuarta articulación con el valor de Q4 en grados
        ax.text(coords_A0_4[0], coords_A0_4[1], coords_A0_4[2], f'({Px4:.1f}, {Py4:.1f}, {Pz4:.1f}) mm', color='magenta', fontsize=6)   # Agregar anotación con las coordenadas XYZ del efector final

        ax.set_xlim(- 400, 400)                                                                             # Defino el límite del eje X para una mejor visualizacion
        ax.set_ylim(- 400, 400)                                                                             # Defino el límite del eje Y para una mejor visualizacion
        ax.set_zlim(0, 400)                                                                                 # Defino el límite del eje Z para una mejor visualizacion

        ax.plot(x, y, z, label="Cinematica Inversa de brazo robotico 4GDL")                                 # Plotear la línea que representa la trayectoria del brazo robótico

        ax.set_xlabel('Eje X')                                                                              # Etiqueto el eje X
        ax.set_ylabel('Eje Y')                                                                              # Etiqueto el eje Y
        ax.set_zlabel('Eje Z')                                                                              # Etiqueto el eje Z
        ax.legend(loc = ("upper left"),fontsize = 8)                                                        # Añadir la leyenda en la parte superior izquierda con tamaño de fuente 8

        plt.show()                                                                                          # Mostramos el gráfico

        return Q1_grados,Q2_grados,Q3_grados,Q4_grados

    def plot_3D_cinematica_inversa(self, Px4, Py4, Pz4, Q, canvas):

        Q1, Q2, Q3, Q4 = self.Cinematica_Inversa(Px4, Py4, Pz4, Q)
        
        Q1 = radians(Q1)
        Q2 = radians(Q2)
        Q3 = radians(Q3)
        Q4 = radians(Q4)
        
        l1 = 96.000                                                                                         # Longitud l1 en milímetros
        l2 = 13.90                                                                                          # Longitud l2 en milímetros
        l3 = 150                                                                                            # Longitud l3 en milímetros
        l4 = 143.24                                                                                         # Longitud l4 en milímetros
        l5 = 127.42                                                                                         # Longitud l5 en milímetros

        A0_1 = Matrix([[cos(Q1), 0, sin(Q1), l2*cos(Q1)],
                    [sin(Q1), 0, -cos(Q1), l2*sin(Q1)],
                    [0, 1, 0, l1],
                    [0, 0, 0, 1]])

        A1_2 = Matrix([[cos(Q2), -sin(Q2), 0, l3*cos(Q2)],
                    [sin(Q2), cos(Q2), 0, l3*sin(Q2)],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])

        A2_3 = Matrix([[cos(Q3), -sin(Q3), 0, l4*cos(Q3)],
                    [sin(Q3), cos(Q3), 0, l4*sin(Q3)],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])

        A3_4 = Matrix([[cos(Q4), -sin(Q4), 0, l5*cos(Q4)],
                    [sin(Q4), cos(Q4), 0, l5*sin(Q4)],
                    [0, 0, 1, 0],
                    [0, 0, 0, 1]])

        A0_2 = A0_1 * A1_2
        A0_3 = A0_2 * A2_3
        A0_4 = A0_3 * A3_4

        coords_A0_1 = A0_1[:3, 3]
        coords_A0_2 = A0_2[:3, 3]
        coords_A0_3 = A0_3[:3, 3]
        coords_A0_4 = A0_4[:3, 3]

        canvas.ax.cla()

        canvas.ax.scatter(0, 0, 0, color='k', label='Articulacion 1')
        canvas.ax.scatter(coords_A0_1[0], coords_A0_1[1], coords_A0_1[2], color='g', label='Articulación 2')
        canvas.ax.scatter(coords_A0_2[0], coords_A0_2[1], coords_A0_2[2], color='b', label='Articulación 3')
        canvas.ax.scatter(coords_A0_3[0], coords_A0_3[1], coords_A0_3[2], color='r', label='Articulación 4')
        canvas.ax.scatter(coords_A0_4[0], coords_A0_4[1], coords_A0_4[2], color='m', label='Efector')

        x = [0, A0_1[0, 3], A0_2[0, 3], A0_3[0, 3], A0_4[0, 3]]
        y = [0, A0_1[1, 3], A0_2[1, 3], A0_3[1, 3], A0_4[1, 3]]
        z = [0, A0_1[2, 3], A0_2[2, 3], A0_3[2, 3], A0_4[2, 3]]

        Q1_deg = math.degrees(Q1)                                                                                   # Convierto el angulo Q1 a grados
        Q2_deg = math.degrees(Q2)                                                                                   # Convierto el angulo Q2 a grados
        Q3_deg = math.degrees(Q3)                                                                                   # Convierto el angulo Q3 a grados
        Q4_deg = math.degrees(Q4)                                                                                   # Convierto el angulo Q4 a grados

        canvas.ax.text(0, 0, 0, f'Q1: {Q1_deg:.2f}°', color='black', fontsize=6)
        canvas.ax.text(coords_A0_1[0], coords_A0_1[1], coords_A0_1[2], f'Q2: {Q2_deg:.2f}°', color='green', fontsize=6)
        canvas.ax.text(coords_A0_2[0], coords_A0_2[1], coords_A0_2[2], f'Q3: {Q3_deg:.2f}°', color='blue', fontsize=6)
        canvas.ax.text(coords_A0_3[0], coords_A0_3[1], coords_A0_3[2], f'Q4: {Q4_deg:.2f}°', color='red', fontsize=6)
        canvas.ax.text(coords_A0_4[0], coords_A0_4[1], coords_A0_4[2], f'({A0_4[0, 3]:.1f}, {A0_4[1, 3]:.1f}, {A0_4[2, 3]:.1f}) mm', color='magenta', fontsize=6)

        canvas.ax.set_xlim(-400, 400)
        canvas.ax.set_ylim(-400, 400)
        canvas.ax.set_zlim(0, 400)

        canvas.ax.plot(x, y, z, label="Cinematica inversa del brazo robotico 4GDL")
        canvas.ax.set_xlabel('Eje X')
        canvas.ax.set_ylabel('Eje Y')
        canvas.ax.set_zlabel('Eje Z')
        canvas.ax.legend(loc="upper left", fontsize=8)

        canvas.draw()

        return Q1_deg, Q2_deg, Q3_deg, Q4_deg

if __name__ == '__main__':
    app = QApplication(sys.argv)                                                                    # Crea una instancia de la aplicación Qt
    window = KinematicsApp()                                                                        # Crea una instancia de la ventana principal de la aplicación
    window.show()                                                                                   # Muestra la ventana principal de la aplicación
    sys.exit(app.exec_())                                                                           # Inicia el bucle de eventos de la aplicación Qt

